In [2]:
import pandas as pd
from tqdm import tqdm
from utils.evaluator import evaluate
dataset = 'Apache'
output_file = f'outputs/parser/LogBatcher_0shot_32candidate_10batchsize_2000chunksize_full/{dataset}_2k.log_structured.csv'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
# a =evaluate(output_file=output_file, groundtruth_file=groundtruth_file,dataset=dataset)
# print(a)
df1 = pd.read_csv(output_file)
logs1 = df1['EventTemplate'].tolist()
df2 = pd.read_csv(groundtruth_file)
logs2 = df2['EventTemplate'].tolist()

count = 0
length = len(logs1)
for i in tqdm(range(length)):
    if logs1[i] == logs2[i]:
        count += 1
print(count / length)
print(count)


100%|██████████| 51978/51978 [00:00<00:00, 2885086.13it/s]

0.8607680172380623
44741


In [ ]:
from utils.postprocess import correct_single_template

print(correct_single_template('<*> "<*> <*> 1.2.3.4:<*> status: <*> len: <*> time: <*>'))

In [3]:
import re
import pandas as pd
dataset = 'HDFS'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
df = pd.read_csv(groundtruth_file)
# 定义更新规则的函数
def update_content(content):
    # 使用正则表达式匹配并更新内容
    pattern = r'(BLOCK\* NameSystem\.allocateBlock: [^ ]*) ([^ ]*)'
    replacement = r'\1. \2'
    return re.sub(pattern, replacement, content)

# 更新Content列
df['Content'] = df['Content'].apply(update_content)

# 保存修改后的CSV文件
df.to_csv('updated_file.csv', index=False)

In [8]:
import pandas as pd
from tqdm import tqdm
from utils.evaluator import evaluate
dataset = 'BGL'
output_file = f'outputs/parser/LILAC_full/{dataset}_full.log_structured.csv'
# output_file ='outputs/parser/LogBatcher_0shot_32candidate_10batchsize/BGL_2k.log_structured.csv'
groundtruth_file = f'dataset/{dataset}/{dataset}_full.log_structured.csv'
# groundtruth_file = 'dataset/BGL/BGL_2k.log_structured_corrected.csv'
a =evaluate(output_file=output_file, groundtruth_file=groundtruth_file,dataset=dataset)
print(a)




MLA:  0.9746803300440204


4631261it [20:44, 3722.82it/s]  


ED:  0.3391685331489631
NED:  0.9961445201966975


100%|██████████| 320/320 [01:35<00:00,  3.35it/s]


PA:  0.9103974058037325
         BGL: group Accuracy: 0.9104, Message-Level Accuracy: 0.9747, Edit Distance: 0.3392, Normalized Edit Distance: 0.996145
(0.9103974058037325, 0.9746803300440204, 0.3391685331489631, 0.9961445201966975)


LogBatcher_0shot_32candidate_10batchsize_2000chunksize_full_time

In [9]:
import json

import tiktoken
def count_message_tokens(messages, model_name):
    # 根据模型名称加载合适的编码器
    if model_name == "gpt-4":
        encoder = tiktoken.encoding_for_model("gpt-4")
    elif model_name == "gpt-3.5-turbo":
        encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
    else:
        raise ValueError("未知的模型名称")

    # 初始化token计数
    token_count = 0

    # 计算每个消息的token数
    for message in messages:
        role_tokens = encoder.encode(message['role'])
        content_tokens = encoder.encode(message['content'])
        token_count += len(role_tokens) + \
            len(content_tokens) + 4  # 加上特殊的消息分隔符的token数

    return token_count

dataset = 'BGL'

counts_token = 0
counts_message = 0

# 存储解析后的日志列表
message_list = []
# load every message
with open('outputs/cost/cost_full_LILAC_32_3_BGL.json', 'r') as file:
    for line in file:
        if line.strip() == '[':
            list_str = ''
            start_load = True
        if line.strip() == ']':
            list_str += line
            message = json.loads(list_str)
            message_list.append(message)
            start_load = False
        if start_load:
            list_str += line
print(len(message_list))

# remove the same log messages
# def make_hashable(log_list):
#     return tuple(tuple(sorted(d.items())) for d in log_list)
# message_list = list(set(make_hashable(log_list) for log_list in message_list))
# message_list = [list(map(dict, log_list)) for log_list in message_list]
# print(len(message_list))

for message in message_list:
    counts_token += count_message_tokens(message, 'gpt-3.5-turbo')
    counts_message += 1
print(counts_token, counts_token/counts_message)


334
86968 260.3832335329341
